# 2. Создание нейронной сети без использования готовых решений

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г. 

In [2]:
import torch

## 1. Создание нейронов и полносвязных слоев

1.1. Используя операции над матрицами и векторами из библиотеки `torch`, реализовать нейрон с заданными весами `weights` и `bias`. Прогнать вектор `inputs` через нейрон и вывести результат. 

In [ ]:
class Neuron:
  def __init__(self, weights, bias):
    # <создать атрибуты объекта weights и bias>
    self.weights = weights
    self.bias = bias
  
  def forward(self, inputs):
    return (inputs * self.weights).sum() + self.bias


In [ ]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([-0.2, 0.3, -0.5, 0.7])
bias = 3.14

1.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать полносвязный слой с заданными весами `weights` и `biases`. Прогнать вектор `inputs` через слой и вывести результат. 

In [2]:
class Linear:
  def __init__(self, weights, biases):
    self.weights = weights
    self.biases = biases
  
  def forward(self, inputs):
    return torch.mv(self.weights, inputs)
    

In [3]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([[-0.2, 0.3, -0.5, 0.7],
                        [0.5, -0.91, 0.26, -0.5],
                        [-0.26, -0.27, 0.17, 0.87]])

biases = torch.tensor([3.14, 2.71, 7.2])

In [4]:
linear = Linear(weights, biases)

linear.forward(inputs)

tensor([ 1.7000, -2.5400,  3.1900])

## 2. Создание функций активации

2.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации.

In [12]:
class ReLU:
  def forward(self, inputs):
    return inputs.clip(min=0)

relu = ReLU()

relu.forward(torch.tensor([1, -1, -6, 1, 2]))


tensor([1, 0, 0, 1, 2])

2.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров.

In [35]:
class Softmax:
  def forward(self, inputs):
    # <реализовать логику Softmax>
    exp = torch.exp(inputs)
    return exp / exp.sum(dim=1, keepdim=True)

softmax = Softmax()

softmax.forward(torch.tensor([[1, 2, 4, 0], [3, 4, 1., 5]]))

tensor([[0.0414, 0.1125, 0.8310, 0.0152],
        [0.0889, 0.2418, 0.0120, 0.6572]])

In [23]:
x = torch.tensor([[1, 2, 4, 0], [3, 4, 1., 5]]).sum(1, keepdim=True)
x

tensor([[ 7.],
        [13.]])

In [34]:
x.sum(-2)

tensor([20.])

## 3. Создание функции потерь

3.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)

Посчитать значение MSE, трактуя вектор `y` как вектор правильных ответов, а `y_pred`, как вектор предсказаний.

In [37]:
class MSELoss:
  def forward(self, y_pred, y_true):
    return torch.mean((y_pred - y_true)**2) # <реализовать логику MSE>

In [39]:
y_pred = torch.tensor([1, 2, 3], dtype=torch.float32)

y = torch.tensor([2, 3, 4], dtype=torch.float32)


mse = MSELoss()

mse.forward(y_pred, y)

tensor(0.3333)

## 4. Обратное распространение ошибки

4.1 Используя один нейрон и SGD (1 пример за шаг), решите задачу регрессии

In [81]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
X = torch.tensor(X, dtype=torch.float32) # <преобразуйте массивы numpy в тензоры torch с типом torch.float32
y = torch.tensor(y, dtype=torch.float32) # <преобразуйте массивы numpy в тензоры torch с типом torch.float32

In [82]:
coef

array([92.72010351, 81.63856342, 31.04647857, 38.91685895])

[Граф вычислений для этой задачи](https://i.ibb.co/2dhDxZx/photo-2021-02-15-17-18-04.jpg)

4.1.1 Модифицируйте класс `MSELoss` из __2.3.1__, реализовав расчет производной относительно предыдущего слоя


In [83]:
class MSELoss:
  def forward(self, y_pred, y_true):
    return torch.mean((y_pred - y_true)**2) # <реализовать логику MSE>

  def backward(self, y_pred, y_true):
    self.dinput = 2 * (y_pred - y_true) # df/dc


4.1.2. Модифицируйте класс `Neuron` из __2.1.1__:

  1) Сделайте так, чтобы веса нейрона инициализировались из стандартного нормального распределения

  2) Реализуйте расчет градиента относительно весов `weights` и `bias`

In [84]:
class Neuron:
  def __init__(self, n_inputs):
    # <создать атрибуты объекта weights и bias>
    # self.n_inputs = n_inputs
    self.weights = torch.randn(n_inputs)
    self.bias = torch.randn(1)
  
  def forward(self, inputs):
    self.input = inputs
    return (inputs * self.weights).sum() + self.bias # <реализовать логику нейрона>
  
  def backward(self, dvalue):
    # dvalue - значение производной, которое приходит нейрону от следующего слоя сети
    # в данном случае это будет значение df/dc (созданное методом backwards у объекта MSELoss)
    self.dweights = dvalue * self.input # df/dW
    self.dinput = dvalue * self.weights # df/wX
    self.dbias = dvalue # df/db


4.1.3 Допишите цикл для настройки весов нейрона

[SGD](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%BE%D1%85%D0%B0%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B3%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%BF%D1%83%D1%81%D0%BA)

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/dda3670f8a8996a0d3bf80856bb4a166cc8db6d4)

In [66]:
n_inputs = 4 # <размерность элемента выборки >
learning_rate = 0.1 #  скорость обучения
n_epoch = 1000 #  количество эпох

neuron = Neuron(n_inputs)
loss = MSELoss()

losses = []

# print(zip(X, y))
for epoch in range(n_epoch):
  
  for x_example, y_example in zip(X, y):
    # forward pass
    y_pred = neuron.forward(x_example) # <прогон через нейрон>
    curr_loss = loss.forward(y_pred, y_example) # <прогон через функцию потерь>
    losses.append(curr_loss)

    # backprop
    # <вызов методов backward>
    loss.backward(y_pred, y_example)
    neuron.backward(loss.dinput)
    # обратите внимание на последовательность вызовов: от конца к началу
    # print(neuron.weights)
    neuron.weights -= learning_rate * neuron.dweights
    neuron.bias -= learning_rate * neuron.dbias
    # <шаг оптимизации для весов (weights и bias) нейрона>

  if epoch % 5 == 0:
    print(f"epoch {epoch} mean loss {torch.stack(losses).mean()}")

epoch 0 mean loss 1061.6754150390625
epoch 5 mean loss 176.94667053222656
epoch 10 mean loss 96.51636505126953
epoch 15 mean loss 66.3550033569336
epoch 20 mean loss 50.556190490722656
epoch 25 mean loss 40.83384704589844
epoch 30 mean loss 34.24774169921875
epoch 35 mean loss 29.491111755371094
epoch 40 mean loss 25.894634246826172
epoch 45 mean loss 23.079999923706055
epoch 50 mean loss 20.8172550201416
epoch 55 mean loss 18.95857048034668
epoch 60 mean loss 17.404590606689453
epoch 65 mean loss 16.086061477661133
epoch 70 mean loss 14.953239440917969
epoch 75 mean loss 13.969473838806152
epoch 80 mean loss 13.107160568237305
epoch 85 mean loss 12.34511661529541
epoch 90 mean loss 11.666812896728516
epoch 95 mean loss 11.05916690826416
epoch 100 mean loss 10.511683464050293
epoch 105 mean loss 10.015849113464355
epoch 110 mean loss 9.564684867858887
epoch 115 mean loss 9.152413368225098
epoch 120 mean loss 8.774214744567871
epoch 125 mean loss 8.426032066345215
epoch 130 mean loss 8.

4.2 Работа с батчами

4.2.1 Модифицируйте класс `MSELoss` из __3.1__, реализовав расчет производной относительно предыдущего слоя с учетом того, что теперь работа ведется с батчами, а не с индивидуальными примерами
 

In [114]:
class MSELoss:
  def forward(self, y_pred, y_true):
    return torch.mean((y_pred - y_true) ** 2)

  def backward(self, y_pred, y_true):
    self.dinput = 2 * (y_pred - y_true) / len(y_true) # df/dy^


4.2.2. Модифицируйте класс `Neuron` из __4.1.2__:

  1) Реализуйте метод `forward` таким образом, чтобы он мог принимать на вход матрицу (батч) с данными. 

  2) Реализуйте расчет градиента относительно весов `weights` и `bias` с учетом того, что теперь работа ведется с батчами, а не с индивидуальными примерами

In [127]:
class Neuron:
  def __init__(self, n_inputs):
    # <создать атрибуты объекта weights и bias>
    self.weights = torch.randn(n_inputs)
    self.bias = torch.tensor(1)
  
  def forward(self, inputs):
    self.inputs = inputs
    return X @ self.weights.T + self.bias # <реализовать логику нейрона>
  
  def backward(self, dvalue):
    # dvalue - значение градиента, которое приходит нейрону от следующего слоя сети
    # в данном случае это будет градиент L по y^ (созданный методом backwards у объекта MSELoss)
    self.dweights = self.inputs.T.mv(dvalue) # df/dW
    self.dbias = dvalue.sum() # df/db

In [116]:
neuron = Neuron(4)

neuron.forward(torch.randn((2, 4)))

tensor([ 1.5436, -0.7334, -3.5034,  3.7786,  0.3309,  0.6941, -0.6677, -0.5985,
         5.2927, -1.9528,  1.0723,  0.5473,  3.3397,  0.2918,  1.3232,  6.6151,
         1.6719, -0.9794,  4.0265, -3.3048, -2.1791,  1.0269,  3.3949, -2.4096,
         2.4610, -1.9779, -0.7896, -3.5132,  0.2139,  0.8727,  2.8040,  2.0103,
         2.4520,  6.9876, -0.5074,  6.1217,  3.0923,  2.1367,  3.6616,  2.1603,
        -1.8690,  3.1847,  0.7969,  1.7345, -2.0365,  3.2037,  3.4775, -1.2714,
         0.6715,  2.3635, -0.3252, -3.7998,  0.3381,  6.1449, -4.3615,  3.5708,
         4.6176,  1.8989,  2.8456, -2.4962,  0.9244, -2.6139,  3.8309,  7.2379,
         2.3081,  2.6172, -3.0549,  4.5765, -3.3604,  5.2114, -1.1295,  0.1421,
         5.6711,  4.1402,  0.3938, -0.0890, -0.3209, -0.3914, -0.8806, -2.7761,
        -1.1531, -3.1797,  8.5684,  2.1941, -2.6391,  3.6554,  2.5856,  3.5995,
         4.3258,  4.8613, -5.4909, -0.9576,  4.3073, -1.3862, -5.2191,  3.4535,
         5.1568,  1.8934,  1.3001, -0.60

# Лабораторная (домашняя) работа 1.2

## 1. Создание нейронов и полносвязных слоев

1.3 Реализовать полносвязный слой из __2.1.2__ таким образом, чтобы он мог принимать на вход матрицу (батч) с данными. Продемонстрировать работу.
Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.


In [13]:
class Linear:
  def __init__(self, weights, biases):
    # <создать атрибуты объекта weights и biases>
    self.weights = weights
    self.biases = biases
  
  def forward(self, inputs):
    A = []
    for input in inputs:
      A.append(torch.mv(self.weights, input) + self.biases) # <реализовать логику слоя>
    return A

inputs = torch.tensor([[1, 2, 3, 2.5], 
                       [2, 5, -1, 2], 
                       [-1.5, 2.7, 3.3, -0.8]])
weights = torch.randn((3, 4))
biases = torch.randn(3)
# print(weights)

batchlayer = Linear(weights, biases)
batchlayer.forward(inputs)

[tensor([-1.2847,  4.8706,  6.1706]),
 tensor([ 4.6422,  4.6035, -0.7359]),
 tensor([-2.9321, -3.3037, 11.1187])]

1.4 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения). Прогнать вектор `inputs` через слой и вывести результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

In [23]:
class Linear:
  def __init__(self, n_features, n_neurons):
    # <создать атрибуты объекта weights и biases>
    self.weights = torch.normal(0, 1, (n_features, n_neurons))
    self.biases = torch.zeros(1, n_neurons)
  
  def forward(self, inputs):
    res = inputs @ self.weights + self.biases # <реализовать логику слоя>
    return res
  
inputs = torch.tensor([[1, 2, 3, 2.5], 
                       [2, 5, -1, 2], 
                       [-1.5, 2.7, 3.3, -0.8]])
std_batchlayer = Linear(4, 5)
std_batchlayer.forward(inputs)

tensor([[  3.5617,  -0.9918,  -7.8641,   0.4858,   1.1843],
        [  7.0173,   1.1546, -13.5138,  -3.0467,   7.0333],
        [-12.1832,   6.8911,   0.5088,  -1.8770,  -5.0833]])

1.5 Используя решение из __1.4__, создать 2 полносвязных слоя и пропустить матрицу `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выбрать произвольно, количество нейронов во втором слое выбрать так, чтобы результатом прогона являлась матрица (3x7). 

In [30]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])


# создаем два полносвязных слоя
layer1 = Linear(4, 5)
layer2 = Linear(5, 7)

# пропускаем матрицу inputs через два слоя
hidden_layer = layer1.forward(inputs)
output_layer = layer2.forward(hidden_layer)

# выводим результат
print(output_layer)
print(output_layer.shape)

tensor([[ 12.1654,   7.1405,   3.1448,   3.1827,   7.4268, -12.1658,  -0.2828],
        [ 17.3693,  19.0116,  47.5131,  18.5378,  20.5940, -15.5535,  -7.0064],
        [  8.8630,  11.7157, -12.0191,  -7.3838,   8.6876, -18.2906,   0.6435]])
torch.Size([3, 7])


## 2. Создание функций активации

2.3 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации.

In [32]:
class ELU:
  def __init__(self, alpha):
    # <создать атрибут объекта alpha>
    self.alpha = alpha

  def forward(self, inputs):
    # <реализовать логику ReLU>
    return torch.where(inputs < 0, self.alpha * (torch.exp(inputs) - 1), inputs)
  
elu = ELU(alpha=1.0)

inputs = torch.randn((4, 3))  # создаем матрицу размера (4,3), заполненную числами из стандартного нормального распределения
output = elu.forward(inputs)  # применяем ELU к матрице inputs

print(output)  # выводим результат

tensor([[-0.3964,  1.1330, -0.3533],
        [ 0.5806,  1.8933,  1.3552],
        [-0.6124, -0.5413, -0.0195],
        [ 0.3653, -0.9320,  0.2052]])


## 3. Создание функции потерь

3.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию потерь Categorical Cross-Entropy:

<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

Создать полносвязный слой с 3 нейронами и прогнать через него батч `inputs`. Полученный результат пропустить через функцию активации softmax. Посчитать значение CCE, трактуя вектор `y` как вектор правильных ответов.

In [36]:
class CategoricalCrossentropyLoss:
  def forward(self, y_pred, y_true):
    # <реализовать логику CCE>
    return - torch.sum(y_true * torch.log(y_pred))

In [37]:
inputs = torch.tensor([[1, 2, 3, 2.5], 
                        [2, 5, -1, 2], 
                        [-1.5, 2.7, 3.3, -0.8]])
y = torch.tensor([1, 0, 0])

layer = torch.nn.Linear(4, 3)

# Пропускаем батч через слой и функцию активации softmax
softmax = Softmax()
y_pred = softmax.forward(layer(inputs))


# Вычисляем значение функции потерь CCE
criterion = CategoricalCrossentropyLoss()
loss = criterion.forward(y_pred, y)

print(loss)

tensor(7.3771, grad_fn=<NegBackward0>)


3.3 Модифицировать 2.3.1, добавив L2-регуляризацию.

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/d92ca2429275bfdc0474523babbafe014ca8b580)


In [38]:
class MSELossL2:
  def __init__(self, lambda_):
    # <создать атрибут объекта alpha>
    self.alpha = lambda_

  def data_loss(self, y_pred, y_true):
    # <подсчет первого слагаемого из формулы>
    return torch.mean((y_pred - y_true)**2)

  def reg_loss(self, layer):
    # используйте атрибуты объекта layer, в которых хранятся веса слоя
    # <подсчет второго слагаемого из формулы>
    return self.alpha * torch.sum(layer.weight**2)

  def forward(self, y_pred, y_true):
    return self.data_loss(y_pred, y_true) + self.reg_loss(y_pred, y_true)

## 4. Обратное распространение ошибки

4.2 Решите задачу 4.1, используя пакетный градиентный спуск

Вычисления для этой задачи: 
[1](https://i.ibb.co/rmtQT6P/photo-2021-02-15-18-00-43.jpg)
[2](https://i.ibb.co/NmCFVnQ/photo-2021-02-15-18-01-17.jpg)

4.2.3 Допишите цикл для настройки весов нейрона

In [98]:
class Neuron:
  def __init__(self, n_inputs):
    # <создать атрибуты объекта weights и bias>
    # self.n_inputs = n_inputs
    self.weights = torch.randn((n_inputs, 1))
    self.bias = torch.randn(1)
  
  def forward(self, inputs):
    self.input = inputs
    return (inputs * self.weights).sum() + self.bias # <реализовать логику нейрона>
  
  def backward(self, dvalue):
    # dvalue - значение производной, которое приходит нейрону от следующего слоя сети
    # в данном случае это будет значение df/dc (созданное методом backwards у объекта MSELoss)
    self.dweights = dvalue * self.input # df/dW
    self.dinput = dvalue * self.weights # df/wX
    self.dbias = dvalue # df/db


In [100]:
n_inputs = 4 # <размерность элемента выборки >
learning_rate = 0.1 # скорость обучения
n_epoch = 100 # количество эпох

neuron = Neuron(n_inputs)

loss = MSELoss()

# создание матрицы X и вектора y
# X = torch.randn(100, n_inputs)
# y = torch.randn(100)

X, y, coef = make_regression(n_features=n_inputs, n_informative=n_inputs, coef=True, bias=0.5)
X = torch.tensor(X, dtype=torch.float32) # <преобразуйте массивы numpy в тензоры torch с типом torch.float32
y = torch.tensor(y, dtype=torch.float32) # <преобразуйте массивы numpy в тензоры torch с типом torch.float32
# print(X.shape, y.shape)

losses = []

for epoch in range(n_epoch):
    # forward pass
    y_pred = neuron.forward(X) # <прогон через нейрон>
    print(neuron.input.size())
    print(neuron.weights.size())
    curr_loss = loss.forward(y_pred, y) # <прогон через функцию потерь>
    losses.append(curr_loss)

    # backprop
    # <вызов методов backward>
    loss.backward(y_pred, y)
    neuron.backward(loss.dinput)

    # <шаг оптимизации для весов (weights и bias) нейрона>
    neuron.weights -= learning_rate * neuron.dweights.mean(dim=0)
    neuron.bias -= learning_rate * neuron.dbias.mean()

    if epoch % 5 == 0:
        print(f"epoch {epoch} mean loss {curr_loss}")

RuntimeError: The size of tensor a (100) must match the size of tensor b (4) at non-singleton dimension 0

4.3  Используя один полносвязный слой и  пакетный градиетный спуск, решите задачу регрессии из __2.4.1__

4.3.1 Модифицируйте класс `Linear` из __1.4__. ([вычисление градиентов](https://i.ibb.co/kgVR6m6/photo-2021-02-15-21-30-28.jpg))

In [103]:
class Linear:
  def __init__(self, n_features, n_neurons):
    # <создать атрибуты объекта weights и biases>
    self.weights = torch.normal(0, 1, (n_features, n_neurons))
    self.biases = torch.zeros(1, n_neurons)
  
  def forward(self, inputs):
    res = inputs @ self.weights + self.biases # <реализовать логику слоя>
    return res
  

In [104]:
class Linear:
  def __init__(self, n_features, n_neurons):
    self.weights = torch.normal(0, 1, (n_features, n_neurons), requires_grad=True)
    self.biases = torch.zeros(1, n_neurons, requires_grad=True)
  
  def forward(self, inputs):
    res = inputs @ self.weights + self.biases
    return res

  def backward(self, dvalues):
    self.dweights = self.weights.grad
    self.dbiases = self.biases.grad
    self.dinputs = dvalues @ self.weights.t()

4.3.2 Создайте слой с одним нейроном. Используя класс MSELoss из 2.4.2, убедитесь, что модель обучается

In [138]:
import torch
import numpy as np

class Linear:
    def __init__(self, n_features, n_neurons):
        self.weights = torch.normal(0, 1, (n_features, n_neurons), requires_grad=True)
        self.biases = torch.zeros(1, n_neurons, requires_grad=True)

    def forward(self, inputs):
        res = inputs @ self.weights + self.biases
        return res

    def backward(self, dvalues):
        self.dweights = self.weights.grad
        self.dbiases = self.biases.grad
        self.dinputs = dvalues @ self.weights.t()

# MSELoss из 2.4.2
class MSELoss:
    def forward(self, y_pred, y_true):
        return torch.mean((y_pred - y_true) ** 2)

    def backward(self, y_pred, y_true):
        self.dinput = torch.mean(2 * (y_pred - y_true)) # df/dy^


X, y, coef = make_regression(n_features=1, n_informative=1, coef=True, bias=0.5)
X = torch.tensor(X, dtype=torch.float32) # <преобразуйте массивы numpy в тензоры torch с типом torch.float32
y = torch.tensor(y, dtype=torch.float32) # <преобразуйте массивы numpy в тензоры torch с типом torch.float32

4.4 Используя наработки из 2.4, создайте нейросеть и решите задачу регрессии.

Предлагаемая архитектура: 
1. Полносвязный слой с 10 нейронами
2. Активация ReLU
3. Полносвязный слой с 1 нейроном

In [ ]:
X = torch.linspace(-1, 1, 100).view(-1, 1)
y = X.pow(2) + 0.2 * torch.rand(X.size()) 

In [ ]:
class Activation_ReLU:
  def forward(self, inputs):
    self.inputs = inputs
    self.output = inputs.clip(min=0)
    return self.output
  
  def backward(self, dvalues):
    self.dinputs = dvalues.clone()
    self.dinputs[self.inputs <= 0] = 0

In [ ]:
# создание компонентов сети
# fc1 = 
# relu1 = 
# fc2 = 

loss = MSELoss()
lr = 0.02

ys = []
for epoch in range(2001):
  # <forward pass>
  # fc1 > relu1 > fc2 > loss

  data_loss = # <прогон через функцию потерь>

  if epoch % 200 == 0:
    print(f'epoch {epoch} mean loss {data_loss}')
    ys.append(out)
  
  # <backprop> 
  # loss > fc2 > relu1 > fc1

  # <шаг оптимизации для fc1>

  # <шаг оптимизации для fc2>


In [102]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(len(ys), 1, figsize=(10, 40))
for ax, y_ in zip(axs, ys):
  ax.scatter(X.numpy(), y.numpy(), color = "orange")
  ax.plot(X.numpy(), y_.numpy(), 'g-', lw=3)
  ax.set_xlim(-1.05, 1.5)
  ax.set_ylim(-0.25, 1.25)

NameError: name 'ys' is not defined